## Initialization  

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys
sys.path.insert(0, '../src')
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src') ; print(sys.path)
import time
import argparse
import yaml
import types, copy, pprint
from time import sleep
from datetime import datetime
import pandas as pd
import numpy  as np
## Set visible GPU device 
##----------------------------------------------
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# from pynvml import *
from utils import (initialize, init_dataloaders, init_environment, init_wandb, training_initializations, model_initializations, 
                   check_for_resume_training, disp_dataloader_info, disp_info_1, warmup_phase, weight_policy_training, 
                   display_gpu_info, init_dataloaders_by_fold_id, print_separator, print_heading, 
                   timestring, print_loss, get_command_line_args, load_from_pickle) 

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
pd.options.display.width = 132
# torch.set_printoptions(precision=6, threshold=None, edgeitems=None, linewidth=132, profile=None, sci_mode=None)
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train.ipynb"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

## Create Environment

### Parse Input Args  - Read YAML config file - wandb initialization

In [3]:
# input_args = " --config yamls/chembl_3task_train.yaml " \
#              " --resume " \
#              " --exp_id      330i85cg" \
#              " --exp_name    0308_1204" \
#              " --lambda_sparsity  0.01"\
#              " --lambda_sharing   0.01" 
#========================================================================
#              " --no_residual "
#              " --exp_name       0410_1934 " \
#              " --hidden_size   100 100 100 100 100 100" \
#              " --tail_hidden_size  100 " \
#              " --decay_lr_rate      0.75"  \
#              " --decay_lr_freq       20"  \

In [4]:
# synthetic_1task_config = "../yamls/chembl_synt_train_1task.yaml"
# synthetic_3task_config = "../yamls/chembl_synt_train_3task.yaml"
# synthetic_5task_config = "../yamls/chembl_synt_train_5task.yaml"
# synthetic_config_file  = "../yamls/chembl_synt_train.yaml"
# config_file      = "../yamls/chembl_mini_train.yaml"
config_file      = "../yamls/chembl_cb29_train.yaml"
config_file      = "../yamls/chembl_cb29_train_1task.yaml"
batch_size=4098


####   For Resume 

In [5]:
restart_input_args = f" --config  {config_file} " \
             " --exp_desc     weight 105 bch/ep policy 105 bch/ep " \
             " --warmup_epochs        10 "  \
             " --hidden_size         4000 4000 4000 4000 4000"  \
             " --tail_hidden_size    4000 "  \
             " --first_dropout       0.80 "  \
             " --middle_dropout      0.80"  \
             " --last_dropout        0.80 "  \
             " --seed_idx              0 "  \
             f" --batch_size        {batch_size} "  \
             " --task_lr           0.001 "  \
             " --backbone_lr       0.001 "  \
             " --decay_lr_rate       0.3 "  \
             " --decay_lr_freq        10 "  \
             " --policy_lr         0.001 "  \
             " --lambda_sparsity    0.02 "  \
             " --lambda_sharing     0.01 "  \
             " --skip_hidden       False "  \
             " --skip_residual     False "  \
             " --pytorch_threads       4 "  \
             " --cuda_devices          2"   \
             " --gpu_ids               0 "  \
             " --resume"                    \
             " --resume_path        ../../experiments/AdaSparseChem-cb29/4000x5_0624_1938_lr0.001_do0.8_RESUME3/" \
             " --resume_ckpt        model_best_seed_model" \
             " --resume_metrics     metrics_best_seed.pickle" \
             " --exp_id             h05zsolg" \
             " --exp_name           0712_0950" \
             " --folder_sfx         RESUME4"

####  For Initiating 

In [6]:
input_args = f" --config  {config_file} " \
             " --exp_desc     weight 105 bch/ep policy 105 bch/ep " \
             " --warmup_epochs         10 "  \
             " --hidden_size         4000 4000 "  \
             " --tail_hidden_size    4000 "  \
             " --first_dropout       0.80 "  \
             " --middle_dropout      0.80"  \
             " --last_dropout        0.80 "  \
             " --seed_idx              0 "  \
             f" --batch_size        {batch_size} "  \
             " --task_lr           0.001 "  \
             " --backbone_lr       0.001 "  \
             " --decay_lr_rate       0.3 "  \
             " --decay_lr_freq        10 "  \
             " --policy_lr         0.001 "  \
             " --lambda_sparsity    0.02 "  \
             " --lambda_sharing     0.01 "  \
             " --skip_hidden       False "  \
             " --skip_residual     False "  \
             " --pytorch_threads       4 "  \
             " --cuda_devices          2"   \
             " --gpu_ids               0 "  

### Read Configuration File

In [7]:
ns = types.SimpleNamespace()
input_args = input_args.split() if input_args is not None else input_args
# input_args = restart_input_args.split() 
ns.args = get_command_line_args(input_args, display = True)
os.environ["CUDA_VISIBLE_DEVICES"]=ns.args.cuda_devices


  command line parms : 
------------------------
 config...................  ../yamls/chembl_cb29_train_1task.yaml
 project_name.............  None
 exp_id...................  x2z5les3
 exp_name.................  None
 folder_sfx...............  None
 exp_desc.................  weight 105 bch/ep policy 105 bch/ep
 hidden_sizes.............  [4000, 4000]
 tail_hidden_size.........  [4000]
 warmup_epochs............  10
 training_epochs..........  None
 seed_idx.................  0
 batch_size...............  4098
 first_dropout............  0.8
 middle_dropout...........  0.8
 last_dropout.............  0.8
 backbone_lr..............  0.001
 task_lr..................  0.001
 policy_lr................  0.001
 decay_lr_rate............  0.3
 decay_lr_freq............  10.0
 lambda_sparsity..........  0.02
 lambda_sharing...........  0.01
 cuda_devices.............  2
 gpu_ids..................  [0]
 pytorch_threads..........  4
 skip_residual............  False
 skip_hidden..............

In [8]:
# display_gpu_info()

In [9]:
opt = initialize(ns, build_folders = True)

##################################################
################### READ YAML ####################
##################################################
 Pytorch thread count: 20
 Set Pytorch thread count to : 4
 Pytorch thread count set to : 4


wandb: Currently logged in as: kbardool. Use `wandb login --relogin` to force relogin


 WandB Initialization -----------------------------------------------------------
 PROJECT NAME: AdaSparseChem-cb29-1Task
 RUN ID      : x2z5les3 
 RUN NAME    : 0818_1444
 --------------------------------------------------------------------------------


 log_dir              create folder:  ../../experiments/AdaSparseChem-cb29-1task/4000x2_0818_1444_lr0.001_do0.8
 result_dir           folder exists:  ../../experiments/AdaSparseChem-cb29-1task/4000x2_0818_1444_lr0.001_do0.8
 checkpoint_dir       folder exists:  ../../experiments/AdaSparseChem-cb29-1task/4000x2_0818_1444_lr0.001_do0.8

------------------------------------------------------------------------------------------------------------------------
 experiment name       : 0818_1444 
 experiment id         : x2z5les3 
 folder_name           : 4000x2_0818_1444_lr0.001_do0.8 
 experiment description: weight 105 bch/ep policy 105 bch/ep
 Random seeds          : [88, 45, 50, 100, 44, 48, 2048, 2222, 9999]
 Random  seed used     : 88 

In [10]:
print_heading(f" experiment name       : {opt['exp_name']} \n"
              f" experiment id         : {opt['exp_id']} \n"
              f" folder_name           : {opt['exp_folder']} \n"
              f" experiment description: {opt['exp_description']}\n"
              f" Random seeds          : {opt['seed_list']}\n"
              f" Seed index            : {opt['seed_idx']} \n"
              f" Policy_iter           : {opt['train']['policy_iter']}\n"
              f" Random  seed used     : {opt['random_seed']} \n"
              f" log folder            : {opt['paths']['log_dir']}\n"
              f" checkpoint folder     : {opt['paths']['checkpoint_dir']}\n"
              f" Gpu ids               : {opt['gpu_ids']}\n"
              f" X Split Ratios        : {opt['dataload']['x_split_ratios']}"
              , verbose = True)

------------------------------------------------------------------------------------------------------------------------
 experiment name       : 0818_1444 
 experiment id         : x2z5les3 
 folder_name           : 4000x2_0818_1444_lr0.001_do0.8 
 experiment description: weight 105 bch/ep policy 105 bch/ep
 Random seeds          : [88, 45, 50, 100, 44, 48, 2048, 2222, 9999]
 Seed index            : 0 
 Policy_iter           : best
 Random  seed used     : 88 
 log folder            : ../../experiments/AdaSparseChem-cb29-1task/4000x2_0818_1444_lr0.001_do0.8
 checkpoint folder     : ../../experiments/AdaSparseChem-cb29-1task/4000x2_0818_1444_lr0.001_do0.8
 Gpu ids               : [0]
 X Split Ratios        : [0.725, 0.225, 0.05]
------------------------------------------------------------------------------------------------------------------------ 



### Setup Dataloader

In [11]:
# dldrs = init_dataloaders(opt, verbose = False)
dldrs = init_dataloaders_by_fold_id(opt, verbose = False)
disp_dataloader_info(dldrs)
# folding  = np.load(os.path.join('/mnt/f/Chembl29', opt['dataload']['folding']))
# print(folding.shape)
# print(folding.max())
# for i in zip(dldrs.valset.num_pos, dldrs.valset.num_neg, dldrs.trainset0.num_pos, dldrs.trainset0.num_neg):
#     print(f" {i[0]:4d}  {i[1]:4d}    trianing: {i[2]:4d}   {i[3]:4d}")

 Warmup folds    : [2, 3, 4]
 Weights folds   : [2, 3]
 Policy folds    : [4]
 Validation folds: [1]
-----------------------------------------------------------------
Load label/Y file for task 1 - task group chembl_29_Y_all.npy
----------------------------------------------------------------- 

 Number of non-zero features in ecfp[0]:79

 Task 1 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :     933144 
    Total   -1  Labels :    1679106 
    Total < -1  Labels :          0 
    Total != 0  Labels :    2612250

  Task 1 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sample  : 3552  Y rows with populated labels: 316273  non zero cols: 933144

  Task 1 files post-filtering : 
---------------------------------
X file : # Samples :  254529     # Features per Sample: 32000 
Y file : # Samples :  254529     # Labels per Sample  : 3552  Y rows w

## Setup Model  

In [12]:

environ = init_environment(ns, opt, is_train = True, display_cfg = True)


##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
 device is  cuda:0
--------------------------------------------------
 SparseChem_Backbone  Ver: 1.0 Init() Start 
-------------------------------------------------- 

 layer config        : [1, 1] 
 skip residual layers: False   skip hidden layers  : False
 SparseChem_BackBone() Input Layer  - Input: 32000  output: 4000  non-linearity:<class 'torch.nn.modules.activation.ReLU'>
 Hidden layer 0 - Input: 4000   output:4000
    _make_layer() using block: <class 'models.sparsechem_backbone.SparseChemBlock'>
           input_size: 4000 output_sz: 4000  non_linearity: ReLU() dropout: 0.8 bias: True
           SparseChemBlock.init(): input_size: 4000 output_sz: 4000   non_linearity: ReLU() dropout: 0.8 bias: True
 Final Hidden layer 0 : Input size: 4000   output size:4000
    _make_layer() using block: <class 'models.sparsechem_backbone.Spars

In [13]:
# environ.optimizers['weights'].param_groups[0]
# print(environ.print_configuration())
# print(environ.networks['mtl-net']._arch_parameters)
# print()
# pp.pprint(environ.networks['mtl-net'].__dict__)

In [14]:
# d = environ.get_task_logits(0,verbose=True)
# print(d.data)
# print(type(d))
# print(d.cpu().numpy())

## Initiate / Resume Training Prep

In [15]:
check_for_resume_training(ns, opt, environ, epoch = 0 , iter = 0)

opt['train']['which_iter'] :  warmup
##################################################
######## Initiate Training from scratch  #########
##################################################


In [16]:
# loaded_metrics
# ns.val_metrics
# ns.best_roc_auc

In [17]:
# a = environ.load_checkpoint('warmup_ep_40_seed_0088', path = '../experiments/AdaSparseChem/50x6_0304_1549_plr0.01_sp0.0001_sh0.01/')
# current_iter = environ.load_checkpoint('latest_weights_policy')
# priRant('Evaluating the snapshot saved at %d iter' % current_iter)

## Warmup Training

### Warmup Training Preparation

In [18]:
model_initializations(ns, opt, environ, phase = 'update_weights', policy_learning = False)

 Model optimizers defined . . . policy_learning: False
 Model schedulers defined . . . policy_learning: False
 Metrics CSV file header written . . . 
 Model initializations complete . . . 


In [19]:
# training_initializations(ns, opt, environ, dldrs, warmup = True)
training_initializations(ns, opt, environ, dldrs, warmup_iterations = 200, weight_iterations = 200, policy_iterations = 200, eval_iterations = 50, warmup = True)
# print('-'*80)
# disp_info_1(ns, opt, environ)
print('-'*80)
print(environ.disp_for_excel())

 training preparation: - check for CUDA - cuda available as device id: [0]
sparsechem_env.cuda()
 training preparation: - set print_freq to                                 : 1989 
 training preparation: - set number of batches per warmup training epoch to: 200
 training preparation: - set number of batches per weight training epoch to: 200
 training preparation: - set number of batches per policy training epoch to: 200
 training preparation: - set number of batches per validation to           : 50
 training preparation complete . . .
--------------------------------------------------------------------------------

    folder: 4000x2_0818_1444_lr0.001_do0.8
    layers: 2 [4000, 4000] 
    
    first dropout          : 0.8
    middle dropout         : 0.8
    last dropout           : 0.8
    diff_sparsity_weights  : False
    skip_layer             : 0
    is_curriculum          : False
    curriculum_speed       : 3
    
    task_lr                : 0.001
    backbone_lr            : 0.

In [20]:
print(ns.eval_iters )
print(ns.trn_iters_warmup)
print(ns.trn_iters_weights)
print(ns.trn_iters_policy)

50
200
200
200


In [21]:
# environ.display_trained_policy(ns.current_epoch,out=sys.stdout)
# ns.check_for_improvment_wait = 0
# ns.warmup_epochs = 1
print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warmup_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.warmup_epochs}", verbose = True)

------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  10 - Run epochs 1 to 10
------------------------------------------------------------------------ 



In [22]:

warmup_phase(ns,opt, environ, dldrs, epochs =1, verbose = False, disable_tqdm = False)


----------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  1 - Run epochs 1 to 1
---------------------------------------------------------------------- 

 Ep  | Trunk LR  Heads LR  Polcy LR  Gmbl Tmp |  trn tsk    trn spar    trn shar   trn ttl |   logloss   bceloss  avg prec    aucroc     aucpr    f1_max |  val tsk    val spar    val shar     total | time |
   1 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   2.6400   1.084e-04   0.000e+00    2.6401 |   0.00007   0.60692   0.66804   0.63838   0.62040   0.73266 |   2.6952   1.084e-04   0.000e+00    2.6954 | 32.7 |
Previous best_epoch:     0   best iter:     0   best_accuracy: 0.00000    best ROC auc: 0.00000
Previous best_epoch:     1   best iter:   200   best_accuracy: 0.66804    best ROC auc: 0.63838
 save best metrics to     :  metrics_best.pickle
 save  best checkpoint to :  model_best
 save warmup checkpoint  to :  model_warmup_last_ep_1
 save warmup metrics to     :  metrics_w

In [23]:
print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best ROC AUC   :   {ns.best_roc_auc:.5f}\n"
      f"Best Precision :   {ns.best_accuracy:.5f}\n")
print()

Best Epoch :       1
Best Iteration :   200 
Best ROC AUC   :   0.63838
Best Precision :   0.66804




In [23]:
print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best ROC AUC   :   {ns.best_roc_auc:.5f}\n"
      f"Best Precision :   {ns.best_accuracy:.5f}\n")
print()
pp.pprint(environ.val_metrics['aggregated'])

Best Epoch :       3
Best Iteration :   7977 
Best ROC AUC   :   0.73847
Best Precision :   0.64680


{   'auc_pr': 0.6352389198378419,
    'avg_prec_score': 0.6468014723213944,
    'bceloss': 0.46895614909788785,
    'f1_max': 0.6818574384687757,
    'kappa': 0.20649701229249132,
    'kappa_max': 0.4290075522297055,
    'logloss': 4.419612724810128e-06,
    'p_f1_max': 0.37923643229349757,
    'p_kappa_max': 0.45070939336602256,
    'roc_auc_score': 0.7384655952099859,
    'sc_loss': 0.003507796458433081}


In [24]:
# ns.wandb_run.finish()
# ns.wandb_run.finish()


## Weight & Policy Training

### Weight/Policy Training Preparation

In [25]:
# print(wandb.run)
# init_wandb(ns, opt)

# ns.flag_warmup = True
# num_train_layers = None 
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
# ns.num_train_layers = None

In [26]:
ns.flag = 'update_weights'
model_initializations(ns, opt, environ, phase = ns.flag, policy_learning = True)
# training_initializations(ns, opt, environ, dldrs, phase='update_weights', weight_iterations = 10, policy_iterations = 10, warmup = False)
training_initializations(ns, opt, environ, dldrs, warmup_iterations = 200,  weight_iterations = 200, policy_iterations = 200, eval_iterations = 50, warmup = False)

 Model optimizers defined . . . policy_learning: True
 Model schedulers defined . . . policy_learning: True
 Metrics CSV file header written . . . 
 Model initializations complete . . . 
 training preparation: - check for CUDA - cuda available as device id: [0]
sparsechem_env.cuda()
 policy policy1 is None
 training preparation: - set print_freq to                                 : 1989 
 training preparation: - set number of batches per warmup training epoch to: 200
 training preparation: - set number of batches per weight training epoch to: 200
 training preparation: - set number of batches per policy training epoch to: 200
 training preparation: - set number of batches per validation to           : 50
 training preparation complete . . .


In [27]:
print_heading( f"** {timestring()} \n"
               f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
               f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
               f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
               f"** Take checkpoint and block gradient flow through Policy net", verbose=True)

------------------------------------------------------------------------------------------------------------------------
** 2022-08-18 14:55:00:650886 
** Training epoch: 1 iter: 200   flag: update_weights 
** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)
** Switch from Warm Up training to Alternate training Weights & Policy 
** Take checkpoint and block gradient flow through Policy net
------------------------------------------------------------------------------------------------------------------------ 



In [28]:
print(hasattr(environ, 'policy%d' % (1)))
print(getattr(environ, "policy1"))
print(environ.opt['gpu_ids'])
print(environ.networks['mtl-net'].policys)

True
None
[0]
[]


In [29]:

print(environ.opt['train']['lambda_sparsity'])
print(environ.opt['train']['lambda_sharing'])
print(environ.opt['train']['decay_temp_freq'])

0.02
0.01
16


In [30]:
# ns.training_epochs = 6
environ.opt['train']['lambda_sparsity'] = 0.1
environ.opt['train']['lambda_sharing']  = 0.001
environ.opt['train']['decay_temp_freq'] = 6
print(environ.opt['train']['lambda_sparsity'])
print(environ.opt['train']['lambda_sharing'])
print(environ.opt['train']['decay_temp_freq'])

0.1
0.001
6


In [20]:
ns.training_epochs = 2

In [31]:
# print( f" ns.flag_warmup                 :      {ns.flag_warmup}")
print( f" ns.flag                        :      {ns.flag}")
print( f" num_train_layers               :      {ns.num_train_layers}")
print( f" environ.opt['is_curriculum']   :      {environ.opt['is_curriculum']}")
print( f" environ.opt['curriculum_speed']:      {environ.opt['curriculum_speed']}\n")
print( f" Backbone Initial LR            :      {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            :      {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            :      {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")

print( f" Hard Sampling                  :      {environ.opt['train']['hard_sampling']}\n")

print( f" Sparsity regularization        :      {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization        :      {environ.opt['train']['lambda_sharing']} \n"
       f" Tasks    regularization        :      {environ.opt['train']['lambda_tasks']}   \n\n")

print( f" Gumbel Temp                    :      {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay              :      {environ.opt['train']['decay_temp_freq']} \n") #

print( f" ns.current_epoch               :      {ns.current_epoch}")
print( f" ns.training_epochs             :      {ns.training_epochs} \n") 
print( f" ns.current_iters               :      {ns.current_iter}")  
print( f" Batches in warmup epoch        :      {ns.trn_iters_warmup}")
print( f" Batches in weight epoch        :      {ns.trn_iters_weights}")
print( f" Batches in policy epoch        :      {ns.trn_iters_policy}")
print( f" Batches in validation          :      {ns.eval_iters}")
print( f" num_train_layers               :      {ns.num_train_layers} \n")

 ns.flag                        :      update_weights
 num_train_layers               :      None
 environ.opt['is_curriculum']   :      False
 environ.opt['curriculum_speed']:      3

 Backbone Initial LR            :      0.001000      Current LR : 0.001 
 Tasks    Initial LR            :      0.001000      Current LR : 0.001    
 Policy   Initial LR            :      0.001000      Current LR : 0.001  

 Hard Sampling                  :      False

 Sparsity regularization        :      0.1
 Sharing  regularization        :      0.001 
 Tasks    regularization        :      1   


 Gumbel Temp                    :      4.0000         
 Gumbel Temp decay              :      6 

 ns.current_epoch               :      1
 ns.training_epochs             :      250 

 ns.current_iters               :      200
 Batches in warmup epoch        :      200
 Batches in weight epoch        :      200
 Batches in policy epoch        :      200
 Batches in validation          :      50
 num_train_l

In [32]:
print_loss(ns.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter} ")
print()
environ.display_trained_policy(ns.current_epoch)
environ.display_trained_logits(ns.current_epoch)
print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

[e] Last ep:1  it:200  -  Total Loss: 2.6954     
Task: 2.6952   Sparsity: 1.08384e-04    Sharing: 0.00000e+00 


 ep:    1    softmax     s         
 ----- ----------------- -    
  0    0.5001    0.4999  1
  1    0.5004    0.4996  1


 ep:    1   logits       s         
 ----- ----------------- -    
  0    0.0006    0.0001  1
  1    0.0010   -0.0005  1

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 1       # of epochs to run:  250 -->  epochs 2 to 251
 policy_learning rate : 0.001 
 lambda_sparsity      : 0.1
 lambda_sharing       : 0.001
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 



### Weight/Policy Training

In [33]:
# weight_policy_training(ns, opt, environ, dldrs, epochs = 100)
weight_policy_training(ns, opt, environ, dldrs, epochs = 1,  display_policy = True, disable_tqdm = False)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 1   # of epochs to run:  1 -->  epochs 2 to 2    
 policy_learning rate : 0.001      
 lambda_sparsity      : 0.1
 lambda_sharing       : 0.001 
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 

 Ep  | Trunk LR  Heads LR  Polcy LR  Gmbl Tmp |  trn tsk    trn spar    trn shar   trn ttl |   logloss   bceloss  avg prec    aucroc     aucpr    f1_max |  val tsk    val spar    val shar     total | time |
   2 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   2.5597   5.419e-04   0.000e+00    2.5602 |   0.00007   0.60205   0.66556   0.62739   0.61913   0.72642 |   2.7470   5.419e-04   0.000e+00    2.7476 | 28.7 |
   2 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   2.0411   6.495e-04   0.000e+00    2.04

In [ ]:
environ.display_trained_policy(ns.current_epoch)
environ.display_trained_logits(ns.current_epoch)

### Close WandB run

In [34]:
ns.wandb_run.finish()

auc_pr,█▁
avg_prec_score,█▁
bceloss,█▁
best_accuracy,▁
best_epoch,▁
best_iter,▁
best_roc_auc,▁
epoch,▁█
f1_max,█▁
gumbel_temp,▁▁
kappa,▁█


In [ ]:
# ns.best_epoch = 0
# from utils.notebook_modules import wrapup_phase
# wrapup_phase(ns, opt, environ)

In [ ]:
# environ.opt['train']['policy_lr']       = 0.002
# environ.opt['train']['lambda_sparsity'] = 0.05
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")

print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" 
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}\n") 

print( f" current_iters               : {ns.current_iter}   \n"
       f" current_epochs              : {ns.current_epoch}  \n" 
       f" train_total_epochs          : {ns.training_epochs}\n" 
       f" stop_epoch_training         : {ns.stop_epoch_training}")

## Post Weight + Policy Training Stuff 

In [ ]:
pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)
# environ.num_layers, environ.networks['mtl-net'].num_layers
# environ.batch

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# model_label   = 'model_train_ep_%d_seed_%04d' % (current_epoch, opt['random_seed'])
# metrics_label = 'metrics_train_ep_%d_seed_%04d.pickle' % (current_epoch, opt['random_seed'])
# environ.save_checkpoint(model_label, current_iter, current_epoch) 
# save_to_pickle(environ.val_metrics, environ.opt['paths']['checkpoint_dir'], metrics_label)
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# print_loss(current_iter, environ.losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, trn_losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, environ.val_metrics, title = f"[e] Policy training epoch:{current_epoch}    iter:")

In [ ]:
# environ.losses
# environ.val_metrics

In [ ]:
# environ.batch_data
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, 0, out=[sys.stdout])
# environ.display_parameters()

# with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
#     environ.print_logit_grads('gradients')

# environ_params = environ.get_task_specific_parameters()
# environ_params = environ.get_arch_parameters()
# environ_params = environ.get_backbone_parameters()
# print(environ_params)
# for param in environ_params:
#     print(param.grad.shape, '\n', param.grad)
#     print(param)

In [ ]:
# environ.display_trained_logits(ns.current_epoch)
# environ.display_trained_policy(ns.current_epoch)

# environ.display_test_sample_policy(ns.current_epoch, hard_sampling = True)
# environ.display_train_sample_policy(ns.current_epoch, hard_sampling = True)

In [ ]:
# opt['exp_instance'] = '0218_1358'     
# folder_name=  f"{opt['exp_instance']}_bs{opt['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"
# print()
# opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
# folder_name=  f"{opt['exp_instance']}_bs{opt['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

In [ ]:
# wandb.finish()

In [ ]:
from utils import load_sparse
ecfp     = load_sparse(opt['dataload']['dataroot'], opt['dataload']['x'])
folding  = np.load(os.path.join(opt['dataload']['dataroot'], opt['dataload']['folding']))

print(ecfp.shape, folding.shape)

fold_va = opt['dataload']['fold_va']
idx_tr  = np.where(folding != fold_va)[0]
idx_va  = np.where(folding == fold_va)[0]

In [ ]:
ecfp_tr = ecfp[idx_tr]
ecfp_tr2 = ecfp[idx_tr2]

In [ ]:
ecfp_tr2.shape

In [ ]:
res = (ecfp_tr != ecfp_tr2)
print(res.shape)
print(res[:10,:10].nnz)

In [ ]:
equal = 0
for i in range(14633):
    for j in range(1):
        if (ecfp_tr[i,j] != ecfp_tr2[i,j]):
            print(i, j, '     ', ecfp_tr[i,j], '    !=    ', ecfp_tr2[i,j])
        else:
            equal+=1 
print("total eaqual = ", equal)
# print(ecfp_tr2[0,:100])
# print(ecfp_tr[:10,:10] != ecfp_tr2[:10,:10])

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

In [ ]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

In [ ]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [ ]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [ ]:
environ.schedulers['weights'].get_last_lr()

In [ ]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

In [ ]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

## Check values

In [ ]:
print(" ns.check_for_improvment_wait:  {ns.check_for_improvment_wait}")
print(" ns.curriculum_epochs:          {ns.curriculum_epochs}")

In [ ]:
# pp.pprint(environ.val_metrics)
df = environ.val_metrics['task1']['classification']

In [ ]:
print(df[pd.notna(df.roc_auc_score)])

In [ ]:
df[pd.notna(df.roc_auc_score)].mean()

In [ ]:
# environ.display_trained_policy(ns.current_epoch,out=[sys.stdout])
# environ.num_tasks
# print(environ.get_policy_prob().shape)
# print(environ.val_data['task1'].keys())
# print(environ.val_data['task1']['yc_ind'][0][:40])
# print(environ.val_data['task1']['yc_ind'][1][:40])
# print(environ.val_data['task1']['yc_data'][:40])
# print(environ.val_data['task1']['yc_hat'][:40])
# environ.display_trained_policy(ns.current_epoch,out=[sys.stdout])
# environ.display_trained_logits(ns.current_epoch,out=[sys.stdout])
batch = next(dldrs.warmup_trn_loader)   

In [ ]:
batch.keys()

In [ ]:
print(batch['x_ind'].shape)
print(batch['x_ind'][:,:20])

In [ ]:
print(type(batch['x_data']))
print(batch['x_data'].shape)

In [ ]:
print(len(batch['row_id']))
print(batch['row_id'])

In [ ]:
print(batch['batch_size'])

In [ ]:
# pp.pprint(environ.val_metrics['task1'])
pp.pprint(environ.val_metrics)
# pp.pprint(ns.val_metrics)
# print((environ.val_data['task1']['yc_data']).sum())
# print(len(environ.val_data['task1']['yc_ind'][1]))
# print(len(environ.val_data['task1']['yc_data']))
# print(len(environ.val_data['task1']['yc_hat']))

In [ ]:
type(ns.val_metrics['aggregated'])

In [ ]:
pp.pprint(ns.trn_losses)

In [ ]:
(environ.val_data['task1']['yc_data'][0] == environ.val_data['task1']['yc_data']).all()

In [ ]:
from utils.sparsechem_utils import compute_metrics, aggregate_results
import pandas
cc = compute_metrics(cols   = environ.val_data['task1']['yc_ind'][1], 
                     y_true = environ.val_data['task1']['yc_data'], 
                     y_score= environ.val_data['task1']['yc_hat'] ,
                     num_tasks=100)


In [ ]:
 df   = pd.DataFrame({"task"   : environ.val_data['task1']['yc_ind'][1], 
                      "y_true" : environ.val_data['task1']['yc_data'],  
                      "y_score": environ.val_data['task1']['yc_hat']})

In [ ]:
for task, frame in df.groupby("task", sort=True):
    print(f" task {task}")
    print(frame.head(10))

In [ ]:
# df
df.groupby("task", sort=True).count()

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
print(environ.batch_data['task1']['yc_aggr_weights'])
environ.batch['task1']['aggr_weights']

In [ ]:
c2 = aggregate_results(cc)

In [ ]:
dldrs.trainset0.tasks_weights_list

In [ ]:

# environ.opt['train']['policy_lr'] = 0.01
# opt['train']['policy_lr']         = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['weights'].param_groups)
# print('current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
# print('current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
# print('current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

## Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_all_task_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


## Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

## Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_all_task_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_all_task_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)